# 数据预处理 
    * 静态数据库分析
        . 剔除无关数据、时间格式转换、计算新特征并生成新数据集
    * 动态数据库分析
        . 计算总各指标数量和每月各指标增长率，生成新数据集
    * 合并分析
        

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

## 静态数据预处理

#### 'isFork', 'archivedAt', 'isTemplate', 'licenseInfo', 'repositoryTopics', 'description', 'isDeprecated' 为无关特征，应舍去

In [4]:
# 读取静态基本信息数据集
repositories_label = pd.read_csv('../data/Repositories_with_label.csv')
# 剔除无关数据
columns_to_drop = [  
    'isFork', 'archivedAt', 'isTemplate', 'licenseInfo',   
    'repositoryTopics', 'description', 'isDeprecated'  
]  
repositories_label = repositories_label.drop(columns=columns_to_drop)

print(repositories_label.shape)
repositories_label.head()

(103354, 7)


,nameWithOwner,stargazerCount,createdAt,updatedAt,pushedAt,isArchived,primaryLanguage
0,goincrypto/cryptocom-exchange,63,2020-01-08T15:01:39Z,2023-05-31T14:03:31Z,2023-02-16T01:46:43Z,False,Python
1,shaofengzeng/SuperPoint-Pytorch,105,2021-10-16T10:01:07Z,2023-06-16T09:13:03Z,2023-05-09T09:46:40Z,False,Python
2,HAOLI-TUKL/Multi_Robots_DMPC,50,2020-09-28T13:13:43Z,2023-06-12T13:20:02Z,2021-02-10T21:31:04Z,False,C++
3,LuxDevHQ/Python-Writing-Clean-Code,44,2021-07-15T22:25:02Z,2023-06-01T18:26:24Z,2023-05-28T10:24:45Z,False,NaN
4,graphql-compose/graphql-compose-json,64,2017-11-03T08:26:38Z,2022-03-04T21:22:07Z,2023-01-07T03:51:09Z,False,TypeScript


In [5]:
repositories_label.dtypes

nameWithOwner      object
stargazerCount      int64
createdAt          object
updatedAt          object
pushedAt           object
isArchived           bool
primaryLanguage    object
dtype: object

#### 加入特征【total_lifetime】【active_lifetime】

In [6]:
# 将时间字符串转换为datetime对象，确保不带时区
repositories_label['createdAt'] = pd.to_datetime(repositories_label['createdAt']).dt.tz_localize(None)
repositories_label['updatedAt'] = pd.to_datetime(repositories_label['updatedAt']).dt.tz_localize(None)

# age表示从建库到2023.12.31的天数
end_date = datetime(2023, 12, 31)
repositories_label['total_lifetime'] = (end_date - repositories_label['createdAt']).dt.days  

# lifetime表示从建库到最后一次更新的天数
repositories_label['active_lifetime'] = (repositories_label['updatedAt'] - repositories_label['createdAt']).dt.days  

# 删除createdAt列
repositories_label.drop('createdAt', axis=1, inplace=True)
repositories_label.drop('updatedAt', axis=1, inplace=True)
repositories_label.drop('pushedAt', axis=1, inplace=True)

In [7]:
print(repositories_label.shape)
repositories_label.head()

(103354, 6)


,nameWithOwner,stargazerCount,isArchived,primaryLanguage,total_lifetime,active_lifetime
0,goincrypto/cryptocom-exchange,63,False,Python,1452,1238
1,shaofengzeng/SuperPoint-Pytorch,105,False,Python,805,607
2,HAOLI-TUKL/Multi_Robots_DMPC,50,False,C++,1188,987
3,LuxDevHQ/Python-Writing-Clean-Code,44,False,NaN,898,685
4,graphql-compose/graphql-compose-json,64,False,TypeScript,2248,1582


In [8]:
print(repositories_label.dtypes)
print(repositories_label.isnull().sum())

nameWithOwner      object
stargazerCount      int64
isArchived           bool
primaryLanguage    object
total_lifetime      int64
active_lifetime     int64
dtype: object
nameWithOwner         0
stargazerCount        0
isArchived            0
primaryLanguage    6867
total_lifetime        0
active_lifetime       0
dtype: int64


In [9]:
repositories_label.to_csv('../data/repositories_label.csv', index=False)

print("label data saved to 'repositories_label.csv'")
repositories_label.info()
repositories_label.head()

label data saved to 'repositories_label.csv'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103354 entries, 0 to 103353
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   nameWithOwner    103354 non-null  object
 1   stargazerCount   103354 non-null  int64 
 2   isArchived       103354 non-null  bool  
 3   primaryLanguage  96487 non-null   object
 4   total_lifetime   103354 non-null  int64 
 5   active_lifetime  103354 non-null  int64 
dtypes: bool(1), int64(3), object(2)
memory usage: 4.0+ MB


,nameWithOwner,stargazerCount,isArchived,primaryLanguage,total_lifetime,active_lifetime
0,goincrypto/cryptocom-exchange,63,False,Python,1452,1238
1,shaofengzeng/SuperPoint-Pytorch,105,False,Python,805,607
2,HAOLI-TUKL/Multi_Robots_DMPC,50,False,C++,1188,987
3,LuxDevHQ/Python-Writing-Clean-Code,44,False,NaN,898,685
4,graphql-compose/graphql-compose-json,64,False,TypeScript,2248,1582


# 动态数据预处理

In [10]:
# 读取动态数据集
repositories_statistics = pd.read_csv('../data/Repositories_with_statistics.csv')

print(repositories_statistics.shape)
repositories_statistics.head()

(7643885, 9)


,Unnamed: 0,nameWithOwner,beforeYearMonth,commits,issues,prs,stars,tags,comments
0,0,00-Evan/shattered-pixel-dungeon-gdx,2014-11-01,0,0,3,1,0,0
1,1,00-Evan/shattered-pixel-dungeon-gdx,2014-12-01,0,0,5,1,0,0
2,2,00-Evan/shattered-pixel-dungeon-gdx,2015-02-01,13,1,0,1,0,0
3,3,00-Evan/shattered-pixel-dungeon-gdx,2015-04-01,16,1,0,0,2,2
4,4,00-Evan/shattered-pixel-dungeon-gdx,2015-05-01,2,0,0,0,1,0


In [11]:
print(repositories_statistics.dtypes)
print(repositories_statistics.isnull().sum())

Unnamed: 0          int64
nameWithOwner      object
beforeYearMonth    object
commits             int64
issues              int64
prs                 int64
stars               int64
tags                int64
comments            int64
dtype: object
Unnamed: 0         0
nameWithOwner      0
beforeYearMonth    0
commits            0
issues             0
prs                0
stars              0
tags               0
comments           0
dtype: int64


#### 加入特征【total_commits】【total_issues】【total_prs】【total_stars】【total_tags】【total_comments】

In [90]:
# 计算总提交数、问题数、PR数等
repositories_statistics['total_commits'] = repositories_statistics.groupby('nameWithOwner')['commits'].transform('sum')
repositories_statistics['total_issues'] = repositories_statistics.groupby('nameWithOwner')['issues'].transform('sum')
repositories_statistics['total_prs'] = repositories_statistics.groupby('nameWithOwner')['prs'].transform('sum')
repositories_statistics['total_stars'] = repositories_statistics.groupby('nameWithOwner')['stars'].transform('sum')
repositories_statistics['total_tags'] = repositories_statistics.groupby('nameWithOwner')['tags'].transform('sum')
repositories_statistics['total_comments'] = repositories_statistics.groupby('nameWithOwner')['comments'].transform('sum')


#### 加入特征【average_commits】【average_issues】【average_prs】【average_stars】【average_tags】【average_comments】

In [91]:
print(repositories_statistics.dtypes)
print(repositories_label.dtypes)
repositories_statistics = pd.merge(repositories_statistics, repositories_label[['nameWithOwner', 'total_lifetime']], 
                      on='nameWithOwner', how='left')
repositories_statistics.head

Unnamed: 0          int64
nameWithOwner      object
beforeYearMonth    object
commits             int64
issues              int64
prs                 int64
stars               int64
tags                int64
comments            int64
total_commits       int64
total_issues        int64
total_prs           int64
total_stars         int64
total_tags          int64
total_comments      int64
dtype: object
nameWithOwner      object
stargazerCount      int64
isArchived           bool
primaryLanguage    object
total_lifetime      int64
active_lifetime     int64
dtype: object


<bound method NDFrame.head of          Unnamed: 0                        nameWithOwner beforeYearMonth  \
0                 0  00-Evan/shattered-pixel-dungeon-gdx      2014-11-01   
1                 1  00-Evan/shattered-pixel-dungeon-gdx      2014-12-01   
2                 2  00-Evan/shattered-pixel-dungeon-gdx      2015-02-01   
3                 3  00-Evan/shattered-pixel-dungeon-gdx      2015-04-01   
4                 4  00-Evan/shattered-pixel-dungeon-gdx      2015-05-01   
...             ...                                  ...             ...   
7643880     7643880                zzzprojects/sqlfiddle      2023-02-01   
7643881     7643881                zzzprojects/sqlfiddle      2023-03-01   
7643882     7643882                zzzprojects/sqlfiddle      2023-04-01   
7643883     7643883                zzzprojects/sqlfiddle      2023-05-01   
7643884     7643884                zzzprojects/sqlfiddle      2023-06-01   

         commits  issues  prs  stars  tags  comments  tot

In [92]:
# 将六列除以divisor_column列
repositories_statistics['average_commits'] = repositories_statistics['total_commits'] / repositories_statistics['total_lifetime']
repositories_statistics['average_issues'] = repositories_statistics['total_issues'] / repositories_statistics['total_lifetime']
repositories_statistics['average_prs'] = repositories_statistics['total_prs'] / repositories_statistics['total_lifetime']
repositories_statistics['average_stars'] = repositories_statistics['total_stars'] / repositories_statistics['total_lifetime']
repositories_statistics['average_tags'] = repositories_statistics['total_tags'] / repositories_statistics['total_lifetime']
repositories_statistics['average_comments'] = repositories_statistics['total_comments'] / repositories_statistics['total_lifetime']


In [93]:
# 删除重复的行
repositories_statistics = repositories_statistics.drop_duplicates(subset=['nameWithOwner'])

# 选择需要的列
repositories_statistics = repositories_statistics[['nameWithOwner', 'total_commits', 'total_issues', 'total_prs', 'total_stars', 'total_tags', 'total_comments', 'average_commits', 'average_issues', 'average_prs', 'average_stars', 'average_tags', 'average_comments']]

repositories_statistics.to_csv('../data/repositories_statistics.csv', index=False)

print("Statistics data saved to 'repositories_statistics.csv'")
repositories_statistics.head(20)

Statistics data saved to 'repositories_statistics.csv'


,nameWithOwner,total_commits,total_issues,total_prs,total_stars,total_tags,total_comments,average_commits,average_issues,average_prs,average_stars,average_tags,average_comments
0,00-Evan/shattered-pixel-dungeon-gdx,160,33,16,175,57,71,0.047690,0.009836,0.004769,0.052161,0.016990,0.021162
107,003random/003Recon,52,8,8,303,0,24,0.023308,0.003586,0.003586,0.135814,0.000000,0.010758
174,01alchemist/TurboScript,407,85,58,563,2,298,0.158243,0.033048,0.022551,0.218896,0.000778,0.115863
254,0312birdzhang/huihuisignin,92,0,0,32,1,0,0.033034,0.000000,0.000000,0.011490,0.000359,0.000000
332,035media/tailwindcss-flexbox-order,31,2,2,41,5,9,0.014685,0.000947,0.000947,0.019422,0.002369,0.004263
392,03lafaye/LollipopDrawerToggle,0,0,1,55,0,0,0.000000,0.000000,0.000298,0.016384,0.000000,0.000000
406,04zhujunjie/ZJJPopup,11,0,0,282,0,0,0.011282,0.000000,0.000000,0.289231,0.000000,0.000000
429,05bit/django-smarter,3,6,5,89,8,0,0.000695,0.001390,0.001158,0.020616,0.001853,0.000000
559,06wj/DragonBonesJS,34,3,2,52,2,12,0.011692,0.001032,0.000688,0.017882,0.000688,0.004127
625,07Agarg/Automatic-Exposure-Correction,2,1,0,38,0,0,0.001304,0.000652,0.000000,0.024772,0.000000,0.000000


# 得到初步处理后的数据集

In [100]:
# 处理缺失值
print(repositories_label.isnull().sum())
print(repositories_statistics.isnull().sum())
# 删除带有缺失值的数据行
repositories_label = repositories_label.dropna(subset=['primaryLanguage'])

nameWithOwner      0
stargazerCount     0
isArchived         0
primaryLanguage    0
total_lifetime     0
active_lifetime    0
dtype: int64
nameWithOwner       0
total_commits       0
total_issues        0
total_prs           0
total_stars         0
total_tags          0
total_comments      0
average_commits     0
average_issues      0
average_prs         0
average_stars       0
average_tags        0
average_comments    0
dtype: int64


In [101]:
# 合并数据集
merged_data = pd.merge(repositories_label, repositories_statistics, on='nameWithOwner', how='inner')
merged_data.head()

,nameWithOwner,stargazerCount,isArchived,primaryLanguage,total_lifetime,active_lifetime,total_commits,total_issues,total_prs,total_stars,total_tags,total_comments,average_commits,average_issues,average_prs,average_stars,average_tags,average_comments
0,goincrypto/cryptocom-exchange,63,False,Python,1452,1238,521,23,22,69,25,58,0.358815,0.015840,0.015152,0.047521,0.017218,0.039945
1,shaofengzeng/SuperPoint-Pytorch,105,False,Python,805,607,140,28,1,121,0,102,0.173913,0.034783,0.001242,0.150311,0.000000,0.126708
2,HAOLI-TUKL/Multi_Robots_DMPC,50,False,C++,1188,987,65,3,0,57,0,0,0.054714,0.002525,0.000000,0.047980,0.000000,0.000000
3,graphql-compose/graphql-compose-json,64,False,TypeScript,2248,1582,61,5,49,65,21,55,0.027135,0.002224,0.021797,0.028915,0.009342,0.024466
4,mcuadros/go-rpi-rgb-led-matrix,75,False,Go,2625,2413,37,12,6,82,0,24,0.014095,0.004571,0.002286,0.031238,0.000000,0.009143


In [102]:
# 保存合并后的数据
merged_data.to_csv('../data/repositories_merged.csv', index=False)

print("Merged data saved to 'repositories_merged.csv'")
merged_data.info()

Merged data saved to 'repositories_merged.csv'
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96487 entries, 0 to 96486
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   nameWithOwner     96487 non-null  object 
 1   stargazerCount    96487 non-null  int64  
 2   isArchived        96487 non-null  bool   
 3   primaryLanguage   96487 non-null  object 
 4   total_lifetime    96487 non-null  int64  
 5   active_lifetime   96487 non-null  int64  
 6   total_commits     96487 non-null  int64  
 7   total_issues      96487 non-null  int64  
 8   total_prs         96487 non-null  int64  
 9   total_stars       96487 non-null  int64  
 10  total_tags        96487 non-null  int64  
 11  total_comments    96487 non-null  int64  
 12  average_commits   96487 non-null  float64
 13  average_issues    96487 non-null  float64
 14  average_prs       96487 non-null  float64
 15  average_stars     96487 non-null  float6